<a href="https://colab.research.google.com/github/21JN1A4503/IIT-FMML-ASSIGNMENTS/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%


How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it? **bold text**
Increasing the Percentage of Validation Set:

Pros:
More data for validation can lead to a more reliable estimate of the model's performance. With a larger validation set, you get a better sense of how well the model generalizes to unseen data.
It can help in detecting overfitting more effectively. If the validation accuracy drops significantly when you increase the validation set size, it might indicate that the model was overfitting the training data.
Cons:
You have less data available for training the model. A larger validation set means a smaller training set. If your training dataset is already small, this can lead to underfitting, where the model may not learn complex patterns well.
Computation time and resources may increase, especially during hyperparameter tuning and training, since more data needs to be processed.
Reducing the Percentage of Validation Set:

Pros:

You have more data available for training the model, which can help improve model performance, especially when the initial dataset is small.
Training and hyperparameter tuning may be faster since less data is used for validation.
Cons:

The estimate of the model's performance on unseen data (validation accuracy) becomes less reliable. With a smaller validation set, you may have higher variance in your performance metrics, making it harder to gauge the model's true generalization ability.
Overfitting may be less detectable. If the validation set is too small, it may not effectively capture overfitting, and you may not notice when the model is fitting noise in the data.
The choice of the percentage of data allocated to the validation set is a trade-off between having a reliable estimate of model performance and having enough data for training. It depends on various factors, including the size of your dataset, the complexity of your model, and your computational resources.

Typical splits include using 70-80% of the data for training and 20-30% for validation. However, you should experiment with different splits and cross-validation techniques to find the best balance for your specific problem and dataset. In practice, using techniques like k-fold cross-validation can help mitigate some of the issues associated with changing the validation set percentage by providing multiple validation subsets for each training-validation split.






**How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?**
arge Validation Set and Small Train Set:

Pros:

The validation set's accuracy is likely to be a more reliable estimate of the model's performance because it's based on a larger sample of data.
Overfitting may be more detectable, as the validation set can provide a more stable measure of model performance.
Cons:

With a smaller training set, the model may not learn complex patterns well, and its performance on the test set could suffer.
If the training set is not representative of the overall data distribution, the model may not generalize well.
In this scenario, predicting test set accuracy using the validation set may be more reliable, but the risk of underfitting (the model not learning the underlying patterns) should be considered.

Balanced Train and Validation Sets:

Pros:

A balanced split between the training and validation sets can provide a reasonable trade-off between model learning and reliable performance estimation.
It allows for a relatively stable and representative estimate of model performance.
Cons:

The size of the training set and validation set should be balanced carefully to ensure an adequate amount of data for training and performance estimation.
In this scenario, predicting test set accuracy using the validation set is more likely to provide a reasonable estimate of model performance.

To make the most informed predictions about test set accuracy using the validation set, it's essential to strike a balance between the sizes of the training and validation sets. Additionally, techniques such as k-fold cross-validation can help mitigate some of the limitations associated with the size of the validation set by repeatedly partitioning the data into training and validation subsets and averaging the results over multiple iterations. This helps provide a more robust estimate of model performance.







**What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?**
Small to Medium-sized Datasets (e.g., less than 10,000 samples):

Training Set: Allocate about 70% to 80% of your data to the training set.
Validation Set: Reserve about 20% to 30% for the validation set.
Larger Datasets (e.g., more than 10,000 samples):

Training Set: You can use a smaller proportion, such as 70% or even less, for the training set since you have more data available.
Validation Set: Reserve around 10% to 20% for the validation set.
These guidelines provide a reasonable starting point, but it's essential to adapt them based on the specific characteristics of your data and problem:

Complex Models: If you are training a very complex model with a high risk of overfitting, you may want to allocate a larger portion to the validation set to get a more reliable estimate of generalization performance.

Limited Data: If your dataset is particularly small, you might need to allocate a larger percentage to the validation set to ensure a sufficient sample for performance estimation. However, be cautious about reducing the training set size too much, as it can lead to underfitting.

Cross-Validation: Instead of a fixed train-validation split, consider using k-fold cross-validation, where the data is split into k subsets, and the model is trained and validated k times. This helps mitigate the impact of the specific choice of split percentages and provides a more robust estimate of model performance.

In practice, it's a good idea to experiment with different split percentages and validation techniques to find the configuration that works best for your specific project. Keep in mind that the goal is to balance the need for sufficient training data with the need for a reliable estimate of how well your model generalizes to unseen data.






## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


Does averaging the validation accuracy across multiple splits give more consistent results?
Reduced Variance: When you perform a single train-validation split, the choice of the data in the validation set can introduce variability into your performance estimate. Different splits may lead to different validation accuracies due to the randomness in the data division. Averaging across multiple splits helps reduce this variance by considering multiple subsets of the data for validation.

Better Representation: In a single split, the validation set might not be fully representative of the overall data distribution. By repeatedly shuffling and partitioning the data into different subsets for validation, you ensure that the model's performance is assessed on various subsets of the data. This provides a more comprehensive assessment of how well the model generalizes.

Robustness: Averaging results over multiple splits makes your performance evaluation more robust to outliers or unusual data distributions in a single split. Outliers or peculiarities in one split may have less impact on the overall performance estimate when averaged with results from other splits.

More Confidence: Averaging provides a more reliable estimate of the model's true generalization performance, which can give you greater confidence in your results when making decisions about model selection, hyperparameter tuning, or comparing different algorithms.

Common choices for the number of folds in k-fold cross-validation include 5-fold or 10-fold, although you can choose other values depending on your dataset size and computational resources. Keep in mind that more folds require more computation but can yield more precise estimates.

In summary, averaging validation accuracy across multiple splits, as seen in k-fold cross-validation, is a best practice in machine learning for obtaining stable and dependable performance metrics. It helps mitigate the influence of random data variations and provides a more realistic assessment of your model's ability to generalize to unseen data.






**Does it give more accurate estimate of test accuracy?**
Reduced Overfitting to the Validation Set: When you perform a single train-validation split, there's a risk that the model may overfit to that specific validation set. This means the model could perform unusually well or poorly on that particular data split. With k-fold cross-validation, the model is trained and validated on different subsets of data, which helps to smooth out the effects of overfitting to any one validation set.

Better Assessment of Generalization: By evaluating the model's performance on multiple validation sets (folds) and averaging the results, you obtain a more comprehensive view of how well the model generalizes to different subsets of the data. This helps you make a more informed estimate of the model's true generalization ability.

Reduction of Data Variability: Machine learning models are sensitive to the specific data they are exposed to. Averaging over multiple data splits reduces the impact of random data variability on the estimated performance, leading to a more accurate representation of the model's expected performance on unseen data.

While k-fold cross-validation provides a more accurate estimate of test accuracy compared to a single train-validation split, it's important to remember that it's still an estimate. The actual test accuracy can vary based on the specific characteristics of the test set, which may differ from the validation sets used in cross-validation. However, by using cross-validation, you can make more informed decisions about model selection, hyperparameter tuning, and overall model performance, which can help you achieve better results when applying your model to real-world data.






**What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?**
Lower k (e.g., k = 5): With a lower number of iterations, each fold represents a larger portion of the dataset, and you have fewer opportunities to assess model performance on different subsets of data. This can lead to higher variability in the estimated performance since the choice of validation sets becomes more critical.

Higher k (e.g., k = 10 or more): Increasing the number of iterations results in smaller validation sets, which can reduce the variance in the performance estimate. With more folds, you assess the model's performance on a wider variety of data subsets, leading to a more stable estimate.

Does a Higher Number of Iterations Provide a Better Estimate?

In general, increasing the number of iterations (higher k) can provide a more robust and stable estimate of model performance. It helps reduce the impact of randomness in the data splits and makes the estimate less sensitive to the specific choice of validation sets.

However, there are diminishing returns with higher k values. As you increase k, the computational cost of cross-validation also increases because you're training and validating the model k times. Additionally, with very high k values, the validation sets become extremely small, which may lead to a less precise estimate due to limited data for validation.

The choice of k should strike a balance between obtaining a reliable estimate and computational efficiency. Common choices for k include 5-fold or 10-fold cross-validation, as they provide a good trade-off between accuracy and computational cost. For smaller datasets or when computational resources are limited, you might opt for a higher k value, while for large datasets, a lower k value can still yield a reasonable estimate.

In summary, a higher number of iterations (higher k) generally provides a better estimate of model performance by reducing the variability in the estimate. However, the choice of k should consider the trade-off between accuracy and computational resources, and it should be made based on the characteristics of your dataset and the available computing power.






Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?**bold text**
Benefits of Increasing Iterations (k):

Smoothing Variability: With a larger value of k, you'll create more diverse subsets of data for both training and validation. This can help smooth out the effects of random data fluctuations and reduce the variance in the performance estimate.

Better Performance Estimate: Increasing k can lead to a more reliable and stable estimate of model performance, especially when dealing with limited data.

Limits to the Effectiveness of Increasing k:

Limited Data: If your training dataset is extremely small, increasing k may not fully resolve the issue of data scarcity. The model's ability to learn meaningful patterns may still be constrained by the small training dataset.

Risk of Overfitting to Validation Sets: When k is large, the validation sets become smaller. If k is too high, there is a risk that the model may start overfitting to the validation sets, which would not reflect its true generalization performance.

Computational Cost: As k increases, the computational cost of k-fold cross-validation also rises because you have to train and validate the model k times. This can be a limiting factor, especially when dealing with resource constraints.

Balancing Act: When dealing with a very small training dataset or validation dataset, increasing k can be a useful strategy to obtain a more stable performance estimate. However, you should strike a balance that takes into account computational resources and the risk of overfitting to the validation sets.

In practice, there's no one-size-fits-all answer to how much you should increase k when dealing with small datasets. It depends on the specific dataset, the model's complexity, and the available computational resources. You may need to experiment with different values of k to find the right balance that provides a reasonable estimate of performance while avoiding overfitting and excessive computational cost. Additionally, consider other techniques like data augmentation or using transfer learning with pre-trained models to make the most of limited data.




